

## Objective
1.  Understanding the Spacex DataSet
2.  Loading the dataset  into the corresponding table in a Db2 database
3.  Executing SQL queries to answer assignment questions 


In [1]:
!pip install sqlalchemy==1.3.9


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 65.7 MB/s eta 0:00:00:00:010:01
  Preparing metadata (setup.py) ... done
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.3.9-cp37-cp37m-linux_x86_64.whl size=1159121 sha256=4dc56c122b4a8d7a4dc6072347c9eb04e998e463a851975c2bd5280a655e61fc
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/03/71/13/010faf12246f72dc76b4150e6e599d13a85b4435e06fb9e51f
Successfully built sqlalchemy
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.3.24
    Uninstalling SQLAlchemy-1.3.24:
      Successfully uninstalled SQLAlchemy-1.3.24


### Connecting to the database



In [2]:
%load_ext sql

In [3]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [4]:
!pip install -q pandas==1.1.5

In [5]:
%sql sqlite:///my_data1.db

'Connected: @my_data1.db'

In [6]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/pandas/core/generic.py:2882: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  both result in 0.1234 being formatted as 0.12.


This below code is added to remove blank rows from table**


In [7]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
(sqlite3.OperationalError) table SPACEXTABLE already exists
[SQL: create table SPACEXTABLE as select * from SPACEXTBL where Date is not null]
(Background on this error at: http://sqlalche.me/e/e3q8)






##### Displaying the names of the unique launch sites  in the space mission


In [8]:
%sql SELECT * FROM SPACEXTBL   LIMIT (5)

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-04-06,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-08-12,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,07:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-08-10,00:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-01-03,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [9]:
%%sql
SELECT DISTINCT "Launch_Site"
FROM SPACEXTBL;

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


In [10]:
%%sql SELECT *
FROM SPACEXTBL WHERE "Launch_Site"="CCAFS LC-40" or "Launch_Site"="CCAFS SLC-40"  LIMIT (5)

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-04-06,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-08-12,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,07:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-08-10,00:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-01-03,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


## Total Payload

In [11]:
%%sql
SELECT SUM(PAYLOAD_MASS__KG_) AS TotalPayloadMass
FROM SPACEXTBL
WHERE Customer = 'NASA (CRS)';

 * sqlite:///my_data1.db
Done.


TotalPayloadMass
45596






###  average payload mass carried by booster version F9 v1.1


In [12]:
%%sql
SELECT AVG(PAYLOAD_MASS__KG_) AS AveragePayloadMass
FROM SPACEXTBL
WHERE Booster_Version LIKE '%F9 v1.0%';

 * sqlite:///my_data1.db
Done.


AveragePayloadMass
340.4



###  first succesful landing outcome in ground pad was acheived.




In [13]:
%%sql
SELECT "Date" AS "first succesful landing"
FROM SPACEXTBL
Where Landing_Outcome LIKE "%Succes%" LIMIT(1)

 * sqlite:///my_data1.db
Done.


first succesful landing
2015-12-22



### boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [14]:
%%sql
SELECT DISTINCT(Booster_Version) AS "boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000"
FROM SPACEXTBL
WHERE Landing_Outcome LIKE "%Success%"AND PAYLOAD_MASS__KG_>4000 AND PAYLOAD_MASS__KG_<6000;

 * sqlite:///my_data1.db
Done.


boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1032.1
F9 B4 B1040.1
F9 FT B1031.2
F9 B4 B1043.1
F9 B5 B1046.2
F9 B5 B1047.2
F9 B5 B1048.3






## number of successful and failure mission outcomes


In [15]:
%%sql
SELECT COUNT(Mission_Outcome) AS "number of successful mission outcomes"
FROM SPACEXTBL
WHERE Mission_Outcome LIKE "%Success"


 * sqlite:///my_data1.db
Done.


number of successful mission outcomes
98


In [16]:
%%sql
SELECT COUNT(Mission_Outcome) AS "number of failed mission outcomes"
FROM SPACEXTBL
WHERE Mission_Outcome LIKE "%Failure%"

 * sqlite:///my_data1.db
Done.


number of failed mission outcomes
1





### names of the booster_versions which have carried the maximum payload mass. Use a subquery


In [17]:
%%sql
SELECT Booster_Version
FROM SPACEXTBL
WHERE PAYLOAD_MASS__KG_ = (SELECT MAX(PAYLOAD_MASS__KG_) FROM SPACEXTBL);


 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6




## records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.



In [30]:
%%sql
SELECT
    CASE substr("Date", 6, 2)
        WHEN '01' THEN 'January'
        WHEN '02' THEN 'February'
        WHEN '03' THEN 'March'
        WHEN '04' THEN 'April'
        WHEN '05' THEN 'May'
        WHEN '06' THEN 'June'
        WHEN '07' THEN 'July'
        WHEN '08' THEN 'August'
        WHEN '09' THEN 'September'
        WHEN '10' THEN 'October'
        WHEN '11' THEN 'November'
        WHEN '12' THEN 'December'
        END AS Month_Name,
    Landing_Outcome AS Outcome,
    Booster_Version AS BoosterVersion,
    Launch_Site AS LaunchSite
FROM
    SPACEXTBL
WHERE
    substr("Date", 1, 4) = '2015'
    AND Landing_Outcome LIKE '%Failure%'
ORDER BY
    Month_Name;

 * sqlite:///my_data1.db
Done.


Month_Name,Outcome,BoosterVersion,LaunchSite
April,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40
October,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40






## Ranking the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.


In [20]:
%%sql
SELECT DISTINCT
    Landing_Outcome,
    COUNT(*) AS Outcome_Count
FROM
    SPACEXTBL
WHERE
    Date >= '2010-06-04' AND Date <= '2017-03-20'
GROUP BY
    Landing_Outcome
ORDER BY
    Outcome_Count DESC;


 * sqlite:///my_data1.db
Done.


Landing_Outcome,Outcome_Count
No attempt,10
Success (ground pad),5
Success (drone ship),5
Failure (drone ship),5
Controlled (ocean),3
Uncontrolled (ocean),2
Precluded (drone ship),1
Failure (parachute),1
